In [ ]:
%pip install chromadb
%pip install sentence-transformers


In [ ]:
import pandas as pd
from pprint import pprint
import chromadb
from itertools import groupby
from sentence_transformers import SentenceTransformer  # For generating embeddings






In [ ]:
df = pd.read_json('./archive/dataset/cpu.json')
# print(df.to_string())

cpu_list = pd.DataFrame(df).dropna().drop_duplicates().to_dict(orient="records")
def key_extract_func(d):
    return next(iter(d))
unique_list = [next(group) for key, group in groupby(sorted(cpu_list, key=key_extract_func), key=key_extract_func)]

pprint(cpu_list)

In [ ]:
import json
cpu_ids = []
for cpu in cpu_list:
    if cpu["name"]:
        cpu_ids.append(cpu["name"])
# pprint(cpu_ids)

# Model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

texts = [
    f"{cpu['name']}: {cpu['core_count']} cores, {cpu['core_clock']} GHz base, "
    f"{cpu['boost_clock']} GHz boost, {cpu['graphics']} graphics, ${cpu['price']}"
    for cpu in cpu_list
]

embeddings = model.encode(texts)

cpu_strings = json.dumps(cpu_list, indent=1)
# print(cpu_strings)

chroma_client = chromadb.Client()
collection = chroma_client.get_collection("cpu_specs")
collection.add(ids=cpu_ids, embeddings=embeddings, metadatas=cpu_list)

print("Data successfully added to chromadb")

# def filter_by_brand(cpu_catalog):
#     filter_list = []
#     remaining_items = []
#     search_str = input("please enter an input: ")
#     print(len(cpu_catalog))
#     for item in cpu_catalog:
#         if search_str.lower() in item["name"].lower():
#             filter_list.append(item)
#         else:
#             remaining_items.append(item)
#     return filter_list
# # pprint(filter_by_brand(cpu_list))
# results = filter_by_brand(cpu_list)
# results_to_string = json.dumps(results, indent=1)
# print(results_to_string)

In [ ]:
# def filter_by_graphics(dictionary):
#     isgpu_list = []
#     nogpu_list = []
#     for gpu in dictionary: 
#         gpu_status = gpu.get("graphics")
#         if gpu_status is not None:
#             isgpu_list.append(gpu)
#         else:
#             nogpu_list.append(gpu) 

#     pprint(isgpu_list)
#     return isgpu_list

# gpu_list = filter_by_graphics(results)